Inicialização

In [1]:
import pandas as pd
import numpy as np

Ler o ficheiro

In [2]:
path_to_file = './04-estacoes-selecionadas/medicoes-longo_seleccionadas.csv'

path_to_dir_destination = './05-Indice-Qualidade-Ar/'

#Carrega o ficheiro para um dataframe
df = pd.read_csv(path_to_file)
df

,data,Station,Poluente,Valor
0,2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9
1,2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9
2,2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6
3,2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6
4,2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6
...,...,...,...,...
31054127,2009-12-31 19:00:00,Vermoim,PM2.5,2.0
31054128,2009-12-31 20:00:00,Vermoim,PM2.5,4.0
31054129,2009-12-31 21:00:00,Vermoim,PM2.5,5.0
31054130,2009-12-31 22:00:00,Vermoim,PM2.5,2.0


Converter a coluna da data em tipo datetime

In [3]:
df['data'] = pd.to_datetime(df['data'])
df.set_index('data', inplace= True)
print(df.index)
df

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 01:00:00',
               '2001-01-01 02:00:00', '2001-01-01 03:00:00',
               '2001-01-01 04:00:00', '2001-01-01 05:00:00',
               '2001-01-01 06:00:00', '2001-01-01 07:00:00',
               '2001-01-01 08:00:00', '2001-01-01 09:00:00',
               ...
               '2009-12-31 14:00:00', '2009-12-31 15:00:00',
               '2009-12-31 16:00:00', '2009-12-31 17:00:00',
               '2009-12-31 18:00:00', '2009-12-31 19:00:00',
               '2009-12-31 20:00:00', '2009-12-31 21:00:00',
               '2009-12-31 22:00:00', '2009-12-31 23:00:00'],
              dtype='datetime64[ns]', name='data', length=31054132, freq=None)


,Station,Poluente,Valor
data,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6
...,...,...,...
2009-12-31 19:00:00,Vermoim,PM2.5,2.0
2009-12-31 20:00:00,Vermoim,PM2.5,4.0
2009-12-31 21:00:00,Vermoim,PM2.5,5.0


Agrupar dados por Estação, Poluente e dia para calcular o indice de qualidade do ar diário

In [4]:
df['dia'] = df.index.date
df

,Station,Poluente,Valor,dia
data,,,,
2001-01-01 00:00:00,Alfragide/Amadora,SO2,2.9,2001-01-01
2001-01-01 01:00:00,Alfragide/Amadora,SO2,2.9,2001-01-01
2001-01-01 02:00:00,Alfragide/Amadora,SO2,5.6,2001-01-01
2001-01-01 03:00:00,Alfragide/Amadora,SO2,5.6,2001-01-01
2001-01-01 04:00:00,Alfragide/Amadora,SO2,5.6,2001-01-01
...,...,...,...,...
2009-12-31 19:00:00,Vermoim,PM2.5,2.0,2009-12-31
2009-12-31 20:00:00,Vermoim,PM2.5,4.0,2009-12-31
2009-12-31 21:00:00,Vermoim,PM2.5,5.0,2009-12-31


Função de cálculo do índice de qualidade do ar de acordo com os métodos estatísticos apropriados para cada poluente:
    - NO2, O3 e SO2: Valor máximo registado no dia
    - PM10 e PM2.5: Valor médio diário

In [5]:
# Separar as colunas de agrupamento
group_cols = ['Station', 'Poluente', 'dia']

# Agrupar e aplicar função personalizada
resultados = []

for (station, poluente, dia), grupo in df.groupby(group_cols):
    if len(grupo) < 18:
        continue  # Ignorar se não cumprir 75%
    
    if poluente in ['PM10', 'PM2.5']:
        aqi = grupo['Valor'].mean()
    elif poluente in ['SO2', 'NO2', 'O3']:
        aqi = grupo['Valor'].max()
    else:
        continue  # Poluente não tratado
    
    resultados.append({
        'Station': station,
        'Poluente': poluente,
        'dia': dia,
        'AQI': aqi
    })

# Criar DataFrame final
aqi_diario = pd.DataFrame(resultados)


Intervalos de valores das categorias do índice da qualidade do ar:

| Classificação | Classificação numérica | PM10      | PM2.5    | NO2       | O3        | SO2       |
|---------------|------------------------|-----------|----------|-----------|-----------|-----------|
| Muito Bom     | 0                      | 0-20      | 0-10     | 0-40      | 0-80      | 0-100     |
| Bom           | 1                      | 21-35     | 11-20    | 41-100    | 81-100    | 101-200   |
| Médio         | 2                      | 36-50     | 21-25    | 101-200   | 101-180   | 201-350   |
| Fraco         | 3                      | 51-100    | 26-50    | 201-400   | 181-240   | 351-500   |
| Mau           | 4                      | 101-1200  | 51-800   | 401-1000  | 241-600   | 501-1250  |


Com base na tabela, define intervalos AQI para cada poluente:

In [6]:
def classificar_aqi(poluente, valor):
    if pd.isna(valor):
        return None

    if poluente == 'PM10':
        if valor <= 20: return 0
        elif valor <= 35: return 1
        elif valor <= 50: return 2
        elif valor <= 100: return 3
        elif valor <= 1200: return 4

    elif poluente == 'PM2.5':
        if valor <= 10: return 0
        elif valor <= 20: return 1
        elif valor <= 25: return 2
        elif valor <= 50: return 3
        elif valor <= 800: return 4

    elif poluente == 'NO2':
        if valor <= 40: return 0
        elif valor <= 100: return 1
        elif valor <= 200: return 2
        elif valor <= 400: return 3
        elif valor <= 1000: return 4

    elif poluente == 'O3':
        if valor <= 80: return 0
        elif valor <= 100: return 1
        elif valor <= 180: return 2
        elif valor <= 240: return 3
        elif valor <= 600: return 4

    elif poluente == 'SO2':
        if valor <= 100: return 0
        elif valor <= 200: return 1
        elif valor <= 350: return 2
        elif valor <= 500: return 3
        elif valor <= 1250: return 4

    return None


aqi_diario['Classificacao_AQI'] = aqi_diario.apply(lambda row: classificar_aqi(row['Poluente'], row['AQI']),axis=1)

mapa_classificacao = {
    0: 'Muito Bom',
    1: 'Bom',
    2: 'Médio',
    3: 'Fraco',
    4: 'Mau'
}

aqi_diario['Qualidade do Ar'] = aqi_diario['Classificacao_AQI'].map(mapa_classificacao)

Cálculo do IQA para cada grupo de poluente, por estação e por dia

In [7]:
aqi_diario

,Station,Poluente,dia,AQI,Classificacao_AQI,Qualidade do Ar
0,Afonso III,NO2,2001-11-23,81.3,1.0,Bom
1,Afonso III,NO2,2001-11-24,77.0,1.0,Bom
2,Afonso III,NO2,2001-11-25,88.0,1.0,Bom
3,Afonso III,NO2,2001-11-26,102.3,2.0,Médio
4,Afonso III,NO2,2001-11-27,80.8,1.0,Bom
...,...,...,...,...,...,...
1285535,Ílhavo,SO2,2022-12-27,2.6,0.0,Muito Bom
1285536,Ílhavo,SO2,2022-12-28,3.0,0.0,Muito Bom
1285537,Ílhavo,SO2,2022-12-29,4.7,0.0,Muito Bom
1285538,Ílhavo,SO2,2022-12-30,0.7,0.0,Muito Bom


Guardar valores num dataframe e guardar num CSV

In [8]:
# Faz print de algumas dessas linhas do dataframe
aqi_diario.head()

# Guarda este dataframe com os AQI num ficheiro CSV
aqi_diario.to_csv("aqi_diario.csv", index = False)
